# Adding resveratrol-producing pathways

In [1]:
from cobra.io import read_sbml_model
from cobra import Reaction, Metabolite
import pandas as pd

In [2]:
model_resv_glc = read_sbml_model('../data/models/model_Y8_bigg_complaint.xml')

In [3]:
model_resv_glc

Name,yeastGEM_v8__46__6__46__2
Memory address,0x02ed0e1b8580
Number of metabolites,2748
Number of reactions,4067
Number of groups,106
Objective expression,1.0*GROWTH - 1.0*GROWTH_reverse_c845e
Compartments,"cell envelope, cytoplasm, extracellular, mitochondrion, nucleus, peroxisome, endoplasmic reticulum, Golgi, lipid particle, vacuole, endoplasmic reticulum membrane, vacuolar membrane, Golgi membrane, mitochondrial membrane"


## Model 1: Resveratrol from glucose 

In [4]:
## Add reactions
# Reaction 1 (HA-TAL):L-tyrosine <=> (E).4.coumarate +   NH4+
new_reaction1 = Reaction('HA-TAL') # 3-(o)xopropanoate:(N)ADP+ (o)xidoreductase
coumarate4 = Metabolite(id='coumarate4', compartment='c')
new_reaction1.add_metabolites({model_resv_glc.metabolites.tyr__L_c: -1, 
                               coumarate4: 1, # 3-Oxopropanoate
                               model_resv_glc.metabolites.nh4_c: 1, # Search fpr NH4 ID
                              })
model_resv_glc.add_reactions([new_reaction1])

In [5]:
model_resv_glc.metabolites.coumarate4

Metabolite identifier,coumarate4
Name,
Memory address,0x2ed26644520
Formula,
Compartment,c
In 3 reaction(s),"HA-TAL, C4H, AT-4CL2"


In [6]:
new_reaction2 = Reaction('AT-4CL1')
E4_coumaroyl_CoA = Metabolite(id='e4_coumaroyl_coa', compartment='c')
new_reaction2.add_metabolites({model_resv_glc.metabolites.coumarate4: -1,
                               model_resv_glc.metabolites.atp_c: -1,
                               model_resv_glc.metabolites.coa_c: -1,
                               E4_coumaroyl_CoA: 1,
                               model_resv_glc.metabolites.amp_c: 1,
                               model_resv_glc.metabolites.ppi_c: 1,
                              })
model_resv_glc.add_reactions([new_reaction2])

In [7]:
new_reaction3 = Reaction('VVVST1')
trans_resv = Metabolite(id='trans_resv', compartment='c')
new_reaction3.add_metabolites({model_resv_glc.metabolites.e4_coumaroyl_coa: -1,
                               model_resv_glc.metabolites.h_c: -3,
                               model_resv_glc.metabolites.malcoa_c: -3,
                               model_resv_glc.metabolites.co2_c: 4,
                               model_resv_glc.metabolites.coa_c: 4,
                               trans_resv: 1
                              })
model_resv_glc.add_reactions([new_reaction3])

In [8]:
model_resv_glc.reactions.get_by_id('VVVST1')

Reaction identifier,VVVST1
Name,
Memory address,0x2ed28791280
Stoichiometry,e4_coumaroyl_coa + 3 h_c + 3 malcoa_c --> 4 co2_c + 4 coa_c + trans_resv + 3 H+ + 3 malonyl-CoA --> 4 carbon dioxide + 4 coenzyme A +
GPR,
Lower bound,0.0
Upper bound,1000.0


In [9]:
model_resv_glc.add_boundary(model_resv_glc.metabolites.trans_resv, type='demand')

Reaction identifier,DM_trans_resv
Name,demand
Memory address,0x2ed2877b8e0
Stoichiometry,trans_resv --> -->
GPR,
Lower bound,0
Upper bound,1000.0


In [10]:
from cobra.io import write_sbml_model
write_sbml_model(model_resv_glc,'../data/models/yeast_resv_glc.xml')

from cobra.io import write_sbml_model
write_sbml_model(model,'data/models/yeast_resv_glc.xml')

In [11]:
with model_resv_glc:
    model_resv_glc.objective=model_resv_glc.reactions.VVVST1
    lets_produce_resv= model_resv_glc.optimize().objective_value
lets_produce_resv

0.31037608119146454

In [12]:
model_resv_glc

Name,yeastGEM_v8__46__6__46__2
Memory address,0x02ed0e1b8580
Number of metabolites,2748
Number of reactions,4071
Number of groups,106
Objective expression,1.0*GROWTH - 1.0*GROWTH_reverse_c845e
Compartments,"cell envelope, cytoplasm, extracellular, mitochondrion, nucleus, peroxisome, endoplasmic reticulum, Golgi, lipid particle, vacuole, endoplasmic reticulum membrane, vacuolar membrane, Golgi membrane, mitochondrial membrane"


## Model 2: Resveratrol from coumaric acid from https://doi.org/10.1128/AEM.00609-06

In [13]:
model_resv_cou = read_sbml_model('../data/models/model_Y8_bigg_complaint.xml')

In [14]:
new_reaction2_1 = Reaction('AT-4CL3')
coumarate4 = Metabolite(id='coumarate4', compartment='c')
E4_coumaroyl_CoA = Metabolite(id='e4_coumaroyl_coa', compartment='c')
new_reaction2_1.add_metabolites({coumarate4: -1,
                               model_resv_cou.metabolites.atp_c: -1,
                               model_resv_cou.metabolites.coa_c: -1,
                               E4_coumaroyl_CoA: 1,
                               model_resv_cou.metabolites.amp_c: 1,
                               model_resv_cou.metabolites.ppi_c: 1,
                              })

model_resv_cou.add_reactions([new_reaction2_1])

In [15]:
model_resv_cou.metabolites.coumarate4

Metabolite identifier,coumarate4
Name,
Memory address,0x2ed2990f940
Formula,
Compartment,c
In 3 reaction(s),"AT-4CL2, AT-4CL3, C4H"


In [16]:
new_reaction2_2 = Reaction('STS')
trans_resv = Metabolite(id='trans_resv', compartment='c')
new_reaction2_2.add_metabolites({model_resv_cou.metabolites.e4_coumaroyl_coa: -1,
                               model_resv_cou.metabolites.h_c: -3,
                               model_resv_cou.metabolites.malcoa_c: -3,
                               model_resv_cou.metabolites.co2_c: 4,
                               model_resv_cou.metabolites.coa_c: 4,
                               trans_resv: 1
                              })
model_resv_cou.add_reactions([new_reaction2_2])

Ignoring reaction 'STS' since it already exists.


In [18]:
new_reaction2_3 = Reaction('Ex_coumarate_transport')
coumarate4_e = Metabolite(id='coumarate4_e', compartment='e')
new_reaction2_3.add_metabolites({coumarate4_e: -1,
                               model_resv_cou.metabolites.coumarate4: 1
                              })

model_resv_cou.add_reactions([new_reaction2_3])


In [19]:
model_resv_cou.add_boundary(model_resv_cou.metabolites.coumarate4_e, type='exchange')
model_resv_cou.add_boundary(model_resv_cou.metabolites.coumarate4, type='sink')

Reaction identifier,SK_coumarate4
Name,sink
Memory address,0x2ed2bcc3e20
Stoichiometry,coumarate4 <=> <=>
GPR,
Lower bound,-1000.0
Upper bound,1000.0


In [34]:
#add coumarate to medium
medium=model_resv_cou.medium
medium['Ex_coumarate_transport'] = 1
model_resv_cou.medium = medium
medium

Ex_coumarate_transport does not seem to be an an exchange reaction. Applying bounds anyway.


{'EX_nh4_e': 1000.0,
 'EX_glc__D_e': 1.0,
 'EX_h_e': 1000.0,
 'EX_fe2_e': 1000.0,
 'EX_o2_e': 1000.0,
 'EX_pi_e': 1000.0,
 'EX_k_e': 1000.0,
 'EX_na1_e': 1000.0,
 'EX_so4_e': 1000.0,
 'EX_h2o_e': 1000.0,
 'EX_cl_e': 1000.0,
 'EX_cu2_e': 1000.0,
 'EX_mn2_e': 1000.0,
 'EX_zn2_e': 1000.0,
 'EX_mg2_e': 1000.0,
 'EX_ca2_e': 1000.0,
 'EX_coumarate4_e': 1000.0,
 'Ex_coumarate_transport': 1}

In [35]:
#how much resveratrol is produced?
model_resv_cou.add_boundary(model_resv_cou.metabolites.trans_resv, type='demand')

ValueError: Boundary reaction 'DM_trans_resv' already exists.

In [36]:
from cobra.io import write_sbml_model
write_sbml_model(model_resv_cou,'../data/models/yeast_resv_cou.xml')

In [37]:
model_resv_cou

Name,yeastGEM_v8__46__6__46__2
Memory address,0x02ed28778130
Number of metabolites,2749
Number of reactions,4072
Number of groups,106
Objective expression,1.0*GROWTH - 1.0*GROWTH_reverse_c845e
Compartments,"cell envelope, cytoplasm, extracellular, mitochondrion, nucleus, peroxisome, endoplasmic reticulum, Golgi, lipid particle, vacuole, endoplasmic reticulum membrane, vacuolar membrane, Golgi membrane, mitochondrial membrane"


## Model 3: Resveratrol from Phenylalanine (https://doi.org/10.1016/j.ymben.2009.07.004)


In [24]:
model_resv_phen = read_sbml_model('../data/models/model_Y8_bigg_complaint.xml')

In [25]:
new_reaction3_1 = Reaction('PAL3')
e_cinnamate_c = Metabolite(id='e_cinnamate_c', compartment='c')
new_reaction3_1.add_metabolites({model_resv_phen.metabolites.phe__L_c: -1,
                               e_cinnamate_c: 1,
                               model_resv_phen.metabolites.nh4_c: 1,
                              })
model_resv_phen.add_reactions([new_reaction3_1])

Ignoring reaction 'PAL3' since it already exists.


In [26]:
new_reaction3_2 = Reaction('C4H') 
coumarate4 = Metabolite(id='coumarate4', compartment='c')
new_reaction3_2.add_metabolites({model_resv_phen.metabolites.e_cinnamate_c: -1,
                               model_resv_phen.metabolites.fmnh2_c: -1,
                               model_resv_phen.metabolites.o2_c: -1,
                               coumarate4: 1, 
                               model_resv_phen.metabolites.h2o_c: 1,
                               model_resv_phen.metabolites.h_c: 1,
                               model_resv_phen.metabolites.fmn_c: 1,
                              })
model_resv_phen.add_reactions([new_reaction3_2])

Ignoring reaction 'C4H' since it already exists.


In [27]:
new_reaction3_3 = Reaction('AT-4CL2')
E4_coumaroyl_CoA = Metabolite(id='e4_coumaroyl_coa', compartment='c')
new_reaction3_3.add_metabolites({model_resv_phen.metabolites.coumarate4: -1,
                               model_resv_phen.metabolites.atp_c: -1,
                               model_resv_phen.metabolites.coa_c: -1,
                               E4_coumaroyl_CoA: 1,
                               model_resv_phen.metabolites.amp_c: 1,
                               model_resv_phen.metabolites.ppi_c: 1,
                              })
model_resv_phen.add_reactions([new_reaction3_3])

Ignoring reaction 'AT-4CL2' since it already exists.


In [28]:
new_reaction3_4 = Reaction('STS')
trans_resv = Metabolite(id='trans_resv', compartment='c')
new_reaction3_4.add_metabolites({model_resv_phen.metabolites.e4_coumaroyl_coa: -1,
                               model_resv_phen.metabolites.h_c: -3,
                               model_resv_phen.metabolites.malcoa_c: -3,
                               model_resv_phen.metabolites.co2_c: 4,
                               model_resv_phen.metabolites.coa_c: 4,
                               trans_resv: 1
                              })
model_resv_phen.add_reactions([new_reaction3_4])

Ignoring reaction 'STS' since it already exists.


In [29]:
from cobra.io import write_sbml_model
write_sbml_model(model_resv_phen,'../data/models/model_Y8_bigg_complaint.xml')

In [30]:
model_resv_phen.add_boundary(model_resv_phen.metabolites.trans_resv, type='demand')

Reaction identifier,DM_trans_resv
Name,demand
Memory address,0x2ed2ec00610
Stoichiometry,trans_resv --> -->
GPR,
Lower bound,0
Upper bound,1000.0


### How much resveratrol is porduced?

In [31]:
# Model using glucose as subsrate
with model_resv_glc:
    model_resv_glc.objective=model_resv_glc.reactions.VVVST1
    lets_produce_resv= model_resv_glc.optimize().objective_value
lets_produce_resv

0.31037608119146454

In [32]:
# Model using phenylananine
with model_resv_phen:
    model_resv_phen.objective=model_resv_phen.reactions.STS
    lets_produce_resv= model_resv_phen.optimize().objective_value
lets_produce_resv

0.29613417745844295

In [38]:
# model usin coumaric acid as substrate
with model_resv_cou:
    model_resv_cou.objective=model_resv_cou.reactions.STS
    lets_produce_resv= model_resv_cou.optimize().objective_value
lets_produce_resv

0.6797348710022589